In [20]:
import os
import sys

In [21]:
import openai
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import DirectoryLoader, CSVLoader, PyPDFLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_community.llms import OpenAI
from langchain_community.vectorstores import Chroma
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_text_splitters import CharacterTextSplitter

In [22]:
import constants
import warnings
import os

In [23]:
os.chdir("/Users/hngh7483/Dropbox/FoxAI/chatgpt")
os.environ["OPENAI_API_KEY"] = constants.APIKEY

In [24]:
loaders = {
    '.pdf': PyPDFLoader,
    '.csv': CSVLoader,
}

In [25]:
def create_directory_loader(file_type, directory_path):
    return DirectoryLoader(
        path=directory_path,
        glob=f"**/*{file_type}",
        loader_cls=loaders[file_type],
        loader_kwargs={"extract_images": False}
    )

In [26]:

# Enable to save to disk & reuse the model (for repeated queries on the same data)
PERSIST = False
query = None
if len(sys.argv) > 1:
  query = sys.argv[1]

In [27]:
if PERSIST and os.path.exists("persist"):
  print("Reusing index...\n")
  vectorstore = Chroma(persist_directory="persist", embedding_function=OpenAIEmbeddings())
  index = VectorStoreIndexWrapper(vectorstore=vectorstore)
else:
  loader = create_directory_loader('.pdf', '/Users/hngh7483/Dropbox/FoxAI/chatgpt/data/pdf')
  #csv_loader = create_directory_loader('.csv', '/Users/hngh7483/Dropbox/chatgpt/data/csv')
  if PERSIST:
    index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":"persist"}).from_loaders([loader])
  else:
    #index = VectorstoreIndexCreator().from_loaders([loader])
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(loader.load_and_split())
    # load it into Chroma
    db = Chroma.from_documents(docs, OpenAIEmbeddings())
    index = VectorStoreIndexWrapper(vectorstore=db)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 105 0 (offset 0)
Ignoring wrong pointing object 260 0 (offset 0)
Ignoring wrong pointing object 739 0 (offset 0)
Ignoring wrong pointing object 1550 0 (offset 0)
Ignoring wrong pointing object 1624 0 (offset 0)


In [28]:
print(index)
print(db.embeddings)

vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x147c0d5b0>
client=<openai.resources.embeddings.Embeddings object at 0x147c8b700> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x14788a250> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [29]:
memory = ConversationBufferMemory(
    memory_key='chat_history', return_messages=True, output_key='answer')

In [30]:
chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model="gpt-4o"),
        retriever=index.vectorstore.as_retriever(search_kwargs={"k": 2 }),
        return_source_documents=True,
        return_generated_question=True,
        memory=memory)

**Câu hỏi trong local document**

In [42]:
chat_history = []
query = "cách tạo mới master item"
#while True:
  #if not query:
   # query = input("Prompt: ")
  #if query in ['quit', 'q', 'exit']:
  #  sys.exit()
result = chain({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

print(result['answer'])

Để tạo một mục mới trong master item, người dùng cần điền các thông tin cần thiết sau:

1. **Mã hàng hóa**: 
   - Có hai lựa chọn để đặt mã hàng hóa:
     - **Item**: Chương trình sẽ tự động tạo mã vật tư theo số mặc định tăng dần; người dùng không có quyền can thiệp.
     - **Manual**: Người dùng có thể gõ mã hàng hóa theo ý muốn.

2. **Tên hàng hóa**: Miêu tả tên của item hoặc hàng hóa.

3. **Item type**: Chọn loại hình của hàng hóa, với các lựa chọn: items, labor, và travel.

4. **Item Group**: Lựa chọn nhóm Item. Nhóm của Item đã được định nghĩa trước và việc thiết lập các thông số như phương pháp lập kế hoạch mua hàng, nguồn cung ứng, số lượng tối thiểu, thời gian hàng về kho, phương pháp mặc định giá, và các tài khoản kế toán sử dụng cho các nghiệp vụ được mặc định theo nhóm Item này.

Thực hiện theo các bước trên sẽ giúp người dùng tạo một mục mới trong master item.


**Ví dụ về câu hỏi kiến thức chung**

In [43]:
query = "các công cụ phân tích tài chính cơ bản"
result = chain({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))
print(result['answer'])



Tôi không có thông tin cụ thể về các công cụ phân tích tài chính từ tài liệu hiện có. Tuy nhiên, một số công cụ phân tích tài chính cơ bản thường bao gồm:

1. **Phân tích tỷ lệ (Ratio Analysis)**: Dùng để đánh giá tình hình tài chính của một công ty thông qua các tỷ lệ tài chính như tỷ lệ thanh khoản, tỷ lệ đòn bẩy, tỷ lệ sinh lời, và tỷ lệ hoạt động.

2. **Phân tích xu hướng (Trend Analysis)**: Theo dõi sự thay đổi trong dữ liệu tài chính qua các kỳ để dự đoán xu hướng tương lai.

3. **Dự báo tài chính (Financial Forecasting)**: Dự đoán hiệu quả tài chính trong tương lai dựa trên dữ liệu lịch sử.

4. **Phân tích dòng tiền (Cash Flow Analysis)**: Đánh giá dòng tiền vào và ra để quản lý dòng tiền hiệu quả nhằm đảm bảo khả năng thanh khoản.

5. **Phân tích điểm hòa vốn (Break-even Analysis)**: Xác định mức doanh thu cần thiết để trang trải tất cả các chi phí trong doanh nghiệp.

Để có thông tin chi tiết hơn hoặc cụ thể về các công cụ phân tích tài chính khác, bạn có thể tham khảo thêm từ